In [2]:
import os
import random
import math
import pandas as pd
import numpy as np
import pickle
import h5py
from csv import writer
from datetime import datetime
import pytz
tzInfo = pytz.timezone('Europe/Paris')

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Dense, Conv1D, Flatten, Dropout, \
                                    MaxPooling1D, Bidirectional, LSTM, Input, \
                                    Activation
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
import keras_tuner as kt

import matplotlib.pyplot as plt

from Models.HyperCDBLSTM import HyperCDBLSTM
from Models.KerasTunerHyperModel_Phase1 import KerasTunerHyperModel

from utils.Logger import Logger
from utils.DataPreparation import walk_forward, prepare_data
from utils.Evaluation import evaluate, plot_training, plot_accuracy, plot_loss
from utils.Experiments import Data, DataDomainwise, Settings, TrainOnce, PretrainingFinetuning, DomainAdversarialLearning, set_seed

tf.version.VERSION

'2.6.0'

In [3]:
project_dir = "../../3_Results/Hyperparametertuning"
project_name = "Phase_1"
project_path = project_dir + "/" + project_name + "/"
if not os.path.exists(project_path):
    os.mkdir(project_path)
    os.mkdir(project_path + "tensorboard/")  
    os.mkdir(project_path + "keras_tuner/")  
    os.mkdir(project_path + "evaluations/")  
else:
    print("Existing directory found.")
if os.path.exists(project_path + "logfile.log"):
    os.rename(project_path + "logfile.log", project_path + "logfile_{}.log".format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")))
logger = Logger(project_dir, project_name, "logfile.log")
logger.activate_logging()

Existing directory found.
Logging to ../../3_Results/Hyperparametertuning/Phase_1/logfile.log


In [8]:
# Load data samples from pickle file
with open('../../1_Data/data_samples_for_hyperparametertuning.pkl', 'rb') as file:
    data_samples = pickle.load(file)

data_samples.keys()

dict_keys([15, 30, 60, 'raw'])

In [9]:
def build_model(hp, oracle):

    #seed = len(oracle._tried_so_far) + 1
    seed = len([t for t in os.listdir(project_path + 'keras_tuner') if 'trial_' in t])
    print("seed:", seed)
    batch_size     = hp.Choice('batch_size', KerasTunerHyperModel.choices['batch_size'])
    window_size    = hp.Choice('window_size', KerasTunerHyperModel.choices['window_size'])
    optimizer      = hp.Choice('optimizer', KerasTunerHyperModel.choices['optimizer'])
    stateful_blstm = hp.Boolean('stateful_blstm', default=False)
    
    max_number_of_convolutions = 2
    max_number_of_blstm_layers = 4
    max_number_of_dense_layers = 4
    number_of_convolutions = hp.Int("number_of_convolutions", min_value=1, max_value=max_number_of_convolutions, step=1)
    number_of_blstm_layers = hp.Int("number_of_blstm_layers", min_value=1, max_value=max_number_of_blstm_layers, step=1)
    number_of_dense_layers = hp.Int("number_of_dense_layers", min_value=1, max_value=max_number_of_dense_layers, step=1)
    
    print("window_size: {}, batch_size: {}, optimizer: {}, stateful_blstm: {}".format(
        window_size, batch_size, optimizer, stateful_blstm))
    print("convolutions: {}, blstm layers: {}, dense layers: {}".format(
        number_of_convolutions, number_of_blstm_layers, number_of_dense_layers))
    
    def next_layer(n_previous, rule, step, minimum):
        """To reduce complexity this function returns n, the number of neurons/cells etc. in a subsequent layer 
            based on the previous layer and a rule. There are three alternatives: 
            Subsequent layers are allowed to have the same, half or minimum n."""
        if n_previous == minimum:
            return minimum
        if rule == "same":
            return n_previous
        if rule == "half":
            return math.ceil(n_previous/2/step)*step
        if rule == "min":
            return minimum
    
    filters = []
    kernel_sizes = []
    filters.append(hp.Int(f"conv_filters_1", min_value=50, max_value=200, step=50))
    kernel_sizes.append(hp.Choice(f"kernel_size_1", [3, 5])) # 3x3 or 5x5
    for i in range(2, number_of_convolutions + 1):
        filters.append(hp.Int(f"conv_filters_{i}", min_value=50, max_value=200, step=50, parent_name="number_of_convolutions", parent_values=[j for j in range(i, max_number_of_convolutions+1)]))
        kernel_sizes.append(hp.Choice(f"kernel_size_{i}", [3, 5], parent_name="number_of_convolutions", parent_values=[j for j in range(i, max_number_of_convolutions+1)])) # 3x3 or 5x5
    print("filters:", filters)
    print("kernel sizes:", kernel_sizes)
    
    lstm_cells = []
    lstm_cells.append(hp.Int(f"lstm_cells_1", min_value=50, max_value=500, step=50))
    for i in range(2, number_of_blstm_layers + 1):
        choice = hp.Choice(f"lstm_cells_{i}", ["same", "half", "min"], parent_name="number_of_blstm_layers", parent_values=[j for j in range(i, max_number_of_blstm_layers+1)])
        lstm_cells.append(next_layer(lstm_cells[-1], choice, 50, 50))
    print("lstm cells:", lstm_cells)
        
    dropout_rates = []
    dense_neurons = []
    dense_neurons.append(hp.Int(f"dense_neurons_1", min_value=100, max_value=500, step=100))
    dropout_rates.append(hp.Float(f"dropout_rate_1", min_value=0.1, max_value=0.5, step=0.2))
    for i in range(2, number_of_dense_layers + 1):
        choice_neurons = hp.Choice(f"dense_neurons_{i}", ["same", "half", "min"], parent_name="number_of_dense_layers", parent_values=[j for j in range(i, max_number_of_dense_layers+1)])
        choice_dropout = hp.Choice(f"dropout_rate_{i}", ["same", "half", "min"], parent_name="number_of_dense_layers", parent_values=[j for j in range(i, max_number_of_dense_layers+1)])
        if i <= number_of_dense_layers:
            dense_neurons.append(next_layer(dense_neurons[-1], choice_neurons, 100, 100))
            dropout_rates.append(next_layer(dropout_rates[-1], choice_dropout, 0.3, 0.1))
    dropout_rates = [round(r, 1) for r in dropout_rates]
    print("dense neurons", dense_neurons)
    print("dropout rates:", dropout_rates)
    
    print("building model...")
    model = HyperCDBLSTM(classes=2, features=1, domains=1, 
                         optimizer=optimizer, batch_size=batch_size, window_size=window_size, seed=seed, 
                         stateful_blstm=stateful_blstm, filters=filters, kernel_sizes=kernel_sizes,
                         lstm_cells=lstm_cells, dropout_rates=dropout_rates, dense_neurons=dense_neurons)
    print("model built")
    return model

In [11]:
oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective("Cohens Kappa", "max"),
        max_trials=1000
)

In [ ]:
tuner = KerasTunerHyperModel(
    data_samples = data_samples,
    hypermodel=(lambda hp: build_model(hp, oracle)),
    oracle=oracle,
    overwrite=False,
    directory=project_path,
    project_name='keras_tuner'
)
tuner.objective=kt.Objective("Cohens Kappa", direction="max"),
tuner.directory

In [15]:
print(datetime.now(tz=tzInfo))
tuner.search()

Trial 1000 Complete [00h 01m 47s]
Cohens Kappa: 0.3744660720401437

Best Cohens Kappa So Far: 0.6477687095498647
Total elapsed time: 12h 55m 19s
INFO:tensorflow:Oracle triggered exit
